# 機械学習

In [10]:
import matplotlib
%matplotlib inline

In [11]:
# データのロード
import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [12]:
# 年齢の欠損を埋める

train['Age'] = train['Age'].fillna(train['Age'].median())

train.drop(['Cabin'], axis=1)


In [13]:
target = train["Survived"].values
features = train[["Pclass", "Sex", "Age", "Fare"]].values

In [14]:
from sklearn.preprocessing import OneHotEncoder

# データの確認

In [15]:
from sklearn.model_selection import train_test_split

# 
non_null_data3 = data.dropna()

target = data['Survived'].values
features = dat[['']]
X_train, X_test, y_train, y_test = train_test_split(
    data['data'], data['target'], random_state=0)



AttributeError: 'dict' object has no attribute 'dropna'

In [ ]:
# 適合率-再現率グラフ出力
def prCurve(ax, y_test, y_pred):
    # ある閾値の時の適合率、再現率, 閾値の値を取得
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred[:,1])

    # 0から1まで0.05刻みで○をプロット
    for i in range(21):
        close_point = np.argmin(np.abs(thresholds - (i * 0.05)))
        ax.plot(precisions[close_point], recalls[close_point], 'o')

    # 適合率-再現率曲線
    ax.plot(precisions, recalls)
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')
    ax.set_title('Precision Recall Curve')

# グラフのメイン出力
def mainGraph(gscv, X, X_test):

    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 8))
    fig.subplots_adjust(wspace=0.5, hspace=0.5)

    # Confusion Matrix 出力
    plot_confusion_matrix(gscv.best_estimator_, X, y, ax=axes[0, 0], display_labels=['0: Dead', '1: Survived'])

    # Grid Searchの結果出力
    result = pd.DataFrame(gscv.cv_results_).set_index('params')
    result.sort_values('rank_test_score', ascending=False).head(10)['mean_test_score'].plot.barh(ax=axes[0, 1], grid=True)
    axes[0, 1].set_xlim(result['mean_test_score'].min() * 0.9, result['mean_test_score'].max() * 1.1)

    # Feature Importance 出力(分類器にあれば)
    axes[0, 2].set_title('Feature Importance/ Coef Top 10')
    if hasattr(gscv.best_estimator_, 'feature_importances_'):
        importances = pd.DataFrame({'Importance':gscv.best_estimator_.feature_importances_}, index=X_test.columns)
        importances.sort_values('Importance', ascending=False).head(10).sort_values('Importance', ascending=True).plot.barh(ax=axes[0, 2], grid=True)
    if hasattr(gscv.best_estimator_, 'coef_'):
        importances = pd.DataFrame({'Coef':gscv.best_estimator_.coef_[0]}, index=X_test.columns)
        importances.sort_values('Coef', ascending=False).head(10).sort_values('Coef', ascending=True).plot.barh(ax=axes[0, 2], grid=True)

    # ROC曲線出力
    plot_roc_curve(gscv.best_estimator_, X, y, ax=axes[1, 0]) 
    axes[1, 0].set_title('ROC(Receiver Operating Characteristic) Curve')

    # 適合率-再現率グラフ出力
    y_pred = gscv.best_estimator_.predict_proba(X)
    output_pr_curve(axes[1, 1], y, y_pred)

    # 学習曲線出力
    output_learning_curve(axes[1, 2], X, y, gscv)

    plt.show()